In [1]:
from fancyimpute import MICE
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, SimpleFill
import pandas as pd
import impyute as imp

Using TensorFlow backend.


In [2]:
data = pd.read_excel('OIC_SESRIC_28Aug2018_gender_dis.xlsx')

#Each year is a column. Converts it into a single column with all the years as rows
data=pd.melt(data, id_vars=['Country','Project Category','Indicator','Selected'], 
             value_vars=[2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017], var_name='Year')

#The Colum "Indicator" is pivoted into a column for each indicator
data=pd.pivot_table(data, values='value', index=['Country','Year'], columns=['Selected','Project Category','Indicator'])

#Sort values
data=data.sort_values(by=['Country','Year'])

#selects only the indicator with the appropriate flag (Selected=1)
data=data.xs(1, level='Selected', axis=1)
data

Project Category                  Economic Context                            \
Indicator                 Agriculture, Value Added Construction, Value Added   
Country              Year                                                      
Afghanistan          2008             2.987752e+09              8.704645e+08   
                     2009             3.911459e+09              9.926377e+08   
                     2010             4.462635e+09              1.238954e+09   
                     2011             5.171297e+09              1.394729e+09   
                     2012             5.416548e+09              1.595694e+09   
                     2013             5.331940e+09              1.695054e+09   
                     2014             5.136259e+09              1.877281e+09   
                     2015             4.482955e+09              2.032602e+09   
                     2016             4.663743e+09              1.868076e+09   
                     2017                      NaN                       NaN   
Albania              2008             2.168845e+09              2.037763e+09   
                     2009             2.022753e+09              1.801621e+09   
                     2010             2.141586e+09              1.541795e+09   
                     2011             2.349589e+09              1.639114e+09   
                     2012             2.312028e+09              1.403089e+09   
                     2013             2.499702e+09              1.314275e+09   
                     2014             2.644345e+09              1.139258e+09   
                     2015             2.274186e+09              1.017998e+09   
                     2016             2.356334e+09              1.011491e+09   
                     2017                      NaN                       NaN   
Algeria              2008             1.126326e+10              1.347090e+10   
                     2009             1.282013e+10              1.376587e+10   
                     2010             1.364852e+10              1.605294e+10   
                     2011             1.622224e+10              1.731016e+10   
                     2012             1.833592e+10              1.820006e+10   
                     2013             2.066321e+10              1.977252e+10   
                     2014             2.199332e+10              2.147209e+10   
                     2015             1.921812e+10              1.847029e+10   
                     2016             1.955629e+10              1.818297e+10   
                     2017                      NaN                       NaN   
...                                            ...                       ...   
United Arab Emirates 2008             2.358167e+09              3.181322e+10   
                     2009             2.375327e+09              3.139872e+10   
                     2010             2.212295e+09              3.184186e+10   
                     2011             2.356203e+09              3.153332e+10   
                     2012             2.393213e+09              3.069522e+10   
                     2013             2.511387e+09              3.148985e+10   
                     2014             2.578144e+09              3.329279e+10   
                     2015             2.653873e+09              3.476991e+10   
                     2016             2.795089e+09              3.565215e+10   
                     2017                      NaN                       NaN   
Uzbekistan           2008             5.838681e+09              1.657628e+09   
                     2009             6.306769e+09              2.286684e+09   
                     2010             7.112190e+09              2.555409e+09   
                     2011             8.214056e+09              2.811771e+09   
                     2012             9.199948e+09              3.283468e+09   
                     2013             1.011687e+10              3.754533e+09   


In [3]:
from sklearn import preprocessing as pp
std_tmp=data
std_tmp=pp.StandardScaler().fit_transform(std_tmp)
std_tmp=pd.DataFrame(std_tmp,index=data.index, columns=data.columns)
std_tmp.describe()

Project Category         Economic Context                            \
Indicator        Agriculture, Value Added Construction, Value Added   
count                        5.130000e+02              5.130000e+02   
mean                        -1.731342e-17             -3.324177e-16   
std                          1.000976e+00              1.000976e+00   
min                         -4.954034e-01             -4.669835e-01   
25%                         -4.638582e-01             -4.409567e-01   
50%                         -3.766996e-01             -3.760342e-01   
75%                         -1.535929e-01             -9.131178e-02   
max                          5.293263e+00              6.067496e+00   

Project Category                               \
Indicator        Ease of Doing Business Index   
count                            1.680000e+02   
mean                             5.286776e-18   
std                              1.002990e+00   
min                             -2.406546e+00   
25%                             -8.267193e-01   
50%                              3.001104e-01   
75%                              8.097320e-01   
max                              1.421278e+00   

Project Category                                                   \
Indicator        Employment in Agriculture, % of Total Employment   
count                                                5.700000e+02   
mean                                                 1.340059e-15   
std                                                  1.000878e+00   
min                                                 -1.382340e+00   
25%                                                 -8.641850e-01   
50%                                                 -1.109012e-01   
75%                                                  7.371863e-01   
max                                                  2.064636e+00   

Project Category                                                \
Indicator        Employment in Industry, % of Total Employment   
count                                             5.700000e+02   
mean                                             -7.479397e-17   
std                                               1.000878e+00   
min                                              -1.538272e+00   
25%                                              -7.510864e-01   
50%                                              -9.242109e-02   
75%                                               5.292727e-01   
max                                               3.414152e+00   

Project Category                                                \
Indicator        Employment in Services, % of Total Employment   
count                                             5.700000e+02   
mean                                             -7.417069e-16   
std                                               1.000878e+00   
min                                              -2.117550e+00   
25%                                              -7.037866e-01   
50%                                               4.001330e-03   
75%                                               7.483148e-01   
max                                               2.102132e+00   

Project Category                                              \
Indicator        Exports of goods and services (current US$)   
count                                           5.230000e+02   
mean                                            9.510132e-17   
std                                             1.000957e+00   
min                                            -5.564184e-01   
25%                                            -5.266358e-01   
50%                                            -4.305770e-01   
75%                                            -5.151019e-02   
max                                             4.695589e+00   

Project Category                                                            \
Indicator        Foreign direct investment, net inflows (BoP, curr

In [4]:
std_tmp=data
std_nom_tmp=(std_tmp - std_tmp.min()) / (std_tmp.max() - std_tmp.min())
std_nom_tmp.describe()

Project Category         Economic Context                            \
Indicator        Agriculture, Value Added Construction, Value Added   
count                          513.000000                513.000000   
mean                             0.085582                  0.071465   
std                              0.172920                  0.153184   
min                              0.000000                  0.000000   
25%                              0.005449                  0.003983   
50%                              0.020506                  0.013918   
75%                              0.059048                  0.057491   
max                              1.000000                  1.000000   

Project Category                               \
Indicator        Ease of Doing Business Index   
count                              168.000000   
mean                                 0.628698   
std                                  0.262026   
min                                  0.000000   
25%                                  0.412722   
50%                                  0.707101   
75%                                  0.840237   
max                                  1.000000   

Project Category                                                   \
Indicator        Employment in Agriculture, % of Total Employment   
count                                                  570.000000   
mean                                                     0.401030   
std                                                      0.290364   
min                                                      0.000000   
25%                                                      0.150322   
50%                                                      0.368856   
75%                                                      0.614894   
max                                                      1.000000   

Project Category                                                \
Indicator        Employment in Industry, % of Total Employment   
count                                               570.000000   
mean                                                  0.310610   
std                                                   0.202099   
min                                                   0.000000   
25%                                                   0.158950   
50%                                                   0.291948   
75%                                                   0.417481   
max                                                   1.000000   

Project Category                                                \
Indicator        Employment in Services, % of Total Employment   
count                                               570.000000   
mean                                                  0.501827   
std                                                   0.237193   
min                                                   0.000000   
25%                                                   0.335040   
50%                                                   0.502775   
75%                                                   0.679166   
max                                                   1.000000   

Project Category                                              \
Indicator        Exports of goods and services (current US$)   
count                                             523.000000   
mean                                                0.105944   
std                                                 0.190586   
min                                                 0.000000   
25%                                                 0.005671   
50%                                                 0.023961   
75%                                                 0.096136   
max                                                 1.000000   

Project Category                                                            \
Indicator        Foreign direct investment, net inflows (BoP, curr

In [5]:
std_tmp=data
std_tmp=pp.RobustScaler(quantile_range=(25.0, 75.0)).fit_transform(std_tmp)
std_tmp=pd.DataFrame(std_tmp,index=data.index, columns=data.columns)
std_nom_tmp=(std_tmp - std_tmp.min()) / (std_tmp.max() - std_tmp.min())
std_nom_tmp.describe()

Project Category         Economic Context                            \
Indicator        Agriculture, Value Added Construction, Value Added   
count                          513.000000                513.000000   
mean                             0.085582                  0.071465   
std                              0.172920                  0.153184   
min                              0.000000                  0.000000   
25%                              0.005449                  0.003983   
50%                              0.020506                  0.013918   
75%                              0.059048                  0.057491   
max                              1.000000                  1.000000   

Project Category                               \
Indicator        Ease of Doing Business Index   
count                              168.000000   
mean                                 0.628698   
std                                  0.262026   
min                                  0.000000   
25%                                  0.412722   
50%                                  0.707101   
75%                                  0.840237   
max                                  1.000000   

Project Category                                                   \
Indicator        Employment in Agriculture, % of Total Employment   
count                                                  570.000000   
mean                                                     0.401030   
std                                                      0.290364   
min                                                      0.000000   
25%                                                      0.150322   
50%                                                      0.368856   
75%                                                      0.614894   
max                                                      1.000000   

Project Category                                                \
Indicator        Employment in Industry, % of Total Employment   
count                                               570.000000   
mean                                                  0.310610   
std                                                   0.202099   
min                                                   0.000000   
25%                                                   0.158950   
50%                                                   0.291948   
75%                                                   0.417481   
max                                                   1.000000   

Project Category                                                \
Indicator        Employment in Services, % of Total Employment   
count                                               570.000000   
mean                                                  0.501827   
std                                                   0.237193   
min                                                   0.000000   
25%                                                   0.335040   
50%                                                   0.502775   
75%                                                   0.679166   
max                                                   1.000000   

Project Category                                              \
Indicator        Exports of goods and services (current US$)   
count                                             523.000000   
mean                                                0.105944   
std                                                 0.190586   
min                                                 0.000000   
25%                                                 0.005671   
50%                                                 0.023961   
75%                                                 0.096136   
max                                                 1.000000   

Project Category                                                            \
Indicator        Foreign direct investment, net inflows (BoP, curr

In [6]:
#standarises data

#try using standard scaler from sk-learn v0.20!!!!!!!
std_tmp=data
std_tmp=(std_tmp-std_tmp.min(axis=0))/(std_tmp.max(axis=0)-std_tmp.min(axis=0))
std_tmp_missing=std_tmp
std_tmp.isna().sum().sum()
std_tmp.describe()

Project Category         Economic Context                            \
Indicator        Agriculture, Value Added Construction, Value Added   
count                          513.000000                513.000000   
mean                             0.085582                  0.071465   
std                              0.172920                  0.153184   
min                              0.000000                  0.000000   
25%                              0.005449                  0.003983   
50%                              0.020506                  0.013918   
75%                              0.059048                  0.057491   
max                              1.000000                  1.000000   

Project Category                               \
Indicator        Ease of Doing Business Index   
count                              168.000000   
mean                                 0.628698   
std                                  0.262026   
min                                  0.000000   
25%                                  0.412722   
50%                                  0.707101   
75%                                  0.840237   
max                                  1.000000   

Project Category                                                   \
Indicator        Employment in Agriculture, % of Total Employment   
count                                                  570.000000   
mean                                                     0.401030   
std                                                      0.290364   
min                                                      0.000000   
25%                                                      0.150322   
50%                                                      0.368856   
75%                                                      0.614894   
max                                                      1.000000   

Project Category                                                \
Indicator        Employment in Industry, % of Total Employment   
count                                               570.000000   
mean                                                  0.310610   
std                                                   0.202099   
min                                                   0.000000   
25%                                                   0.158950   
50%                                                   0.291948   
75%                                                   0.417481   
max                                                   1.000000   

Project Category                                                \
Indicator        Employment in Services, % of Total Employment   
count                                               570.000000   
mean                                                  0.501827   
std                                                   0.237193   
min                                                   0.000000   
25%                                                   0.335040   
50%                                                   0.502775   
75%                                                   0.679166   
max                                                   1.000000   

Project Category                                              \
Indicator        Exports of goods and services (current US$)   
count                                             523.000000   
mean                                                0.105944   
std                                                 0.190586   
min                                                 0.000000   
25%                                                 0.005671   
50%                                                 0.023961   
75%                                                 0.096136   
max                                                 1.000000   

Project Category                                                            \
Indicator        Foreign direct investment, net inflows (BoP, curr

In [12]:
#removing data to simulate
#todo!!!! test removing a full column
import random
import numpy as np
import random

std_tmp_missing=std_tmp.copy()
ix = [(row, col) for row in range(std_tmp_missing.shape[0]) for col in range(std_tmp_missing.shape[1])]
for row, col in random.sample(ix, int(round(.1*len(ix)))):
    std_tmp_missing.iat[row, col] = np.nan

#add full inducators in nan for some countries (random)
tmp=pd.DataFrame()
for name,group in std_tmp_missing.groupby("Country"):
    tmp9=group
    t=random.randint(1,25)
    if (t<10):
        t2=random.randint(0,len(group.columns.values))
        while(group.iloc[:,t2].isna().sum()==10):
            print('already')
            t2=random.randint(0,len(group.columns.values))
        else:
            group.iloc[:,t2]=np.nan
            print(group.iloc[:,t2])
        tmp9=group
    
    tmp=tmp.append(tmp9, sort=True)
    
    
print(std_tmp_missing.isna().sum().sum())
print(std_tmp.isna().sum().sum())    


/srv/conda/lib/python3.6/site-packages/pandas/core/indexing.py:630: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


Country     Year
Azerbaijan  2008   NaN
            2009   NaN
            2010   NaN
            2011   NaN
            2012   NaN
            2013   NaN
            2014   NaN
            2015   NaN
            2016   NaN
            2017   NaN
Name: (Human Catalysts for STI, Total R&D personnel (HC) - Private non-profit), dtype: float64
Country  Year
Benin    2008   NaN
         2009   NaN
         2010   NaN
         2011   NaN
         2012   NaN
         2013   NaN
         2014   NaN
         2015   NaN
         2016   NaN
         2017   NaN
Name: (Outputs of STI, Articles Published), dtype: float64
Country  Year
Brunei   2008   NaN
         2009   NaN
         2010   NaN
         2011   NaN
         2012   NaN
         2013   NaN
         2014   NaN
         2015   NaN
         2016   NaN
         2017   NaN
Name: (Social Context, Political Stability and Absence of Violence/Terrorism: Estimate), dtype: float64
Country       Year
Burkina Faso  2008   NaN
              2009   Na

In [ ]:
#mice and after knn
tmp=pd.DataFrame()
for name, group in std_tmp_missing.groupby('Country'):
    print(name)
    tmp2=group.dropna(axis=1,how='all')
    tmp3=MICE(visit_sequence='roman', init_fill_method='mean', n_imputations=50, n_burn_in=5, impute_type='col', verbose=False, min_value=0, max_value=1).complete(tmp2)
    tmp3=pd.DataFrame(tmp3,columns=tmp2.columns, index=tmp2.index)
    tmp=tmp.append(tmp3, sort=True)

#check how to avoid using itself as neighbour!!!!!!!!!!!!!!!!    
mice_knn=KNN(k=5, min_value=0, max_value=1).complete(tmp)
mice_knn=pd.DataFrame(mice_knn, columns=tmp.columns, index=tmp.index)


Afghanistan


In [65]:
#only median
tmp_median=std_tmp_missing
median=SimpleFill(fill_method='median').complete(tmp_median)
median=pd.DataFrame(median, columns=tmp_median.columns, index=tmp_median.index)
median

Project Category                  Economic Context                            \
Indicator                 Agriculture, Value Added Construction, Value Added   
Country              Year                                                      
Afghanistan          2008                 0.023581                  0.008973   
                     2009                 0.020839                  0.010236   
                     2010                 0.035344                  0.012782   
                     2011                 0.040996                  0.014392   
                     2012                 0.042952                  0.016469   
                     2013                 0.042278                  0.017496   
                     2014                 0.040717                  0.019379   
                     2015                 0.035506                  0.020984   
                     2016                 0.020839                  0.019284   
                     2017                 0.020839                  0.012902   
Albania              2008                 0.020839                  0.021038   
                     2009                 0.020839                  0.018597   
                     2010                 0.016832                  0.015912   
                     2011                 0.018491                  0.016917   
                     2012                 0.018191                  0.014478   
                     2013                 0.019688                  0.012902   
                     2014                 0.020842                  0.011751   
                     2015                 0.017889                  0.010498   
                     2016                 0.018545                  0.010431   
                     2017                 0.020839                  0.012902   
Algeria              2008                 0.089585                  0.012902   
                     2009                 0.102002                  0.142254   
                     2010                 0.108609                  0.165892   
                     2011                 0.129137                  0.178886   
                     2012                 0.145995                  0.188084   
                     2013                 0.164557                  0.204336   
                     2014                 0.175166                  0.221902   
                     2015                 0.153031                  0.190877   
                     2016                 0.155729                  0.187907   
                     2017                 0.020839                  0.012902   
...                                            ...                       ...   
United Arab Emirates 2008                 0.018559                  0.328783   
                     2009                 0.018696                  0.324499   
                     2010                 0.017396                  0.329079   
                     2011                 0.020839                  0.012902   
                     2012                 0.018839                  0.317228   
                     2013                 0.019781                  0.325441   
                     2014                 0.020839                  0.344075   
                     2015                 0.020918                  0.359342   
                     2016                 0.022044                  0.368460   
                     2017                 0.020839                  0.012902   
Uzbekistan           2008                 0.046319                  0.017109   
                     2009                 0.050053                  0.023610   
                     2010                 0.056477                  0.026388   
                     2011                 0.065265                  0.029037   
                     2012                 0.073128                  0.033913   
                     2013                 0.080441                  0.038781   


In [66]:
#linear+propagation+median
tpm_l_p_m=std_tmp_missing
l_p_m=tpm_l_p_m.groupby('Country').apply(
    lambda group:group.interpolate(method='linear', limit_area='inside'))
l_p_m=l_p_m.groupby('Country').apply(
    lambda group:group.fillna(method='ffill')) 
l_p_m=l_p_m.groupby('Country').apply(
    lambda group:group.fillna(method='bfill')) 

l_p_m=l_p_m.fillna(l_p_m.median())

In [67]:
#linear+propagation+mean
tpm_l_p_mean=std_tmp_missing
l_p_mean=tpm_l_p_mean.groupby('Country').apply(
    lambda group:group.interpolate(method='linear', limit_area='inside'))
l_p_mean=l_p_mean.groupby('Country').apply(
    lambda group:group.fillna(method='ffill')) 
l_p_mean=l_p_mean.groupby('Country').apply(
    lambda group:group.fillna(method='bfill')) 

#try replacing mean by knn -->>>#check how to avoid using itself as neighbour!!!!!!!!!!!!!!!! 
l_p_mean=l_p_mean.fillna(l_p_m.mean())

In [95]:
#error mice_knn
(((std_tmp-mice_knn)**2).mean(0)**0.5).describe()

count    132.000000
mean       0.023323
std        0.033768
min        0.000000
25%        0.005139
50%        0.012356
75%        0.026371
max        0.264619
dtype: float64

In [69]:
#error median
(((std_tmp-median)**2).mean(0)**0.5).describe()

count    132.000000
mean       0.069059
std        0.047429
min        0.000000
25%        0.042660
50%        0.061048
75%        0.090211
max        0.324443
dtype: float64

In [70]:
#error linear+propagation+median
(((std_tmp-l_p_m)**2).mean(0)**0.5).describe()

count    132.000000
mean       0.022740
std        0.039637
min        0.000000
25%        0.004230
50%        0.011166
75%        0.026056
max        0.324443
dtype: float64

In [71]:
#error linear+propagation+mean
(((std_tmp-l_p_mean)**2).mean(0)**0.5).describe()

count    132.000000
mean       0.021623
std        0.033196
min        0.000000
25%        0.004566
50%        0.011271
75%        0.025944
max        0.234847
dtype: float64

In [ ]:
#try knn and mice
#try other algorithms from impyute

In [ ]:
group.iloc[:,t2]